In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[17]:
 
                                                                 # type: ignore
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import cv2
import tkinter as tk
from tkinter import ttk
import PIL 
import time
import onnxruntime as ort
import scipy.special
#import gpiozero as gpio
#from gpiozero import Button
#capButton = Button(26)
from ultralytics import YOLO
from ultralytics.engine.results import Results
# Updated code (using recommended function)

model = YOLO(r"C:\Users\Carlos Magahis\thesis\best.pt")
#session = ort.InferenceSession(r"C:\Users\Carlos Magahis\thesis\best.pt")


from tkinter import *
from tkinter import filedialog
import PIL.Image, PIL.ImageTk
import os
from tkinter import PhotoImage
root = tk.Tk()
root.geometry("800x480")
root.resizable(width=True, height = False)

color1 =  '#93C572'
class ImageCaptureApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.config(bg=color1)
        self.window.title(window_title)
        self.video_source = 1
        self.vid = cv2.VideoCapture(self.video_source)
        self.vid.set(3, 240)
        self.vid.set(4, 180)
        self.vid.set(cv2.CAP_PROP_AUTOFOCUS, 0) # turn the autofocus off
        self.canvas = tk.Canvas(window, width=self.vid.get(cv2.CAP_PROP_FRAME_WIDTH), height=self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT), bg = color1)
        self.canvas.grid(row = 0, column = 0)
        
        self.label = Label(root, text = " ", bg = color1)
        self.label.grid (row = 0, column = 1)
        
        self.image_width = 560  
        self.image_height = 360 
        self.image_frame = tk.Canvas(window, width=560, height=360, bg = color1)
        self.image_frame.grid(row = 0, column = 2)
        self.image = None  
        
        photo = PhotoImage('button.png')
        self.btn_capture = tk.Button(window, text="Capture", width=10, command=self.capture)
        self.btn_capture.grid(row = 1, column = 0)
        
        self.select_button = tk.Button(root, text="Select Image", command=self.select)
        self.select_button.grid(row = 1, column = 2)
        
        Font_tuple = ("Verdana", 10, "bold")
        self.text_output = tk.Text(window, height=2, width=40)
        self.text_output.grid(row = 2, column = 2)
        self.text_output.configure(font = "Verdana")
      
        #capButton.when_pressed = self.capture
        time.sleep(3)
        
        self.update()
        
        self.window.mainloop()
    
    def capture(self):
        
        ret, frame = self.vid.read()
        time.sleep(6)
        
        timestamp = int(time.time())
        if ret:
            image_path = os.path.join(r'C:\Users\Carlos Magahis\thesis\camera folder',f"image{timestamp}.png" )  
            cv2.imwrite(image_path,frame)
            dispimg = cv2.imread(image_path) 
            input_shape = (1, 3, 640, 640)
            #dispimg = cv2.cvtColor(dispimg, cv2.COLOR_BGR2RGB) 
            dispimg = cv2.cvtColor(dispimg, cv2.COLOR_RGB2BGR) #convert lang nung captured image
            self.image = dispimg
            self.predict_image(image_path)
           
    def predict_image(self, image_path):
        results = model(image_path)
        if not results or not results[0].boxes:
                label = "No Dragonfruit stem detected"
                print(label)
                self.display_prediction(label)  # Update UI with the message
                dispimg = cv2.imread(image_path)
                dispimg = cv2.cvtColor(dispimg, cv2.COLOR_RGB2BGR)
                self.image = dispimg  # Display original image without any changes
                  # Exit if no detections
        found = False
        for result in results:
            boxes = result.boxes
            mask = result.masks
            names = result.names
           
            keypoints = result.keypoints
            probs = result.probs
            print(probs)
            confidence_threshold = 0.5
            obb = result.obb
            
            
            for i, box in enumerate(boxes):
                # Get the confidence score for the detection
                conf = box.conf[0].item()  # Get the maximum probability for the detection
    
                # If the confidence is below 0.5, skip processing this detection
                if conf < 0.5:
                    found = False
                    label = f"No Dragonfruit stem detected"
                    print(f"Detection {i}: Confidence = {conf:.2f} (Too Low) - Skipping this detection.{label}")
                    continue  # Skip to the next detection
                class_idx = box.cls[0].item()
                # Process and handle detections with high confidence
                label = names[class_idx]  # Get the label from the class index
                found = True
                print(f"Detection {i}: Confidence = {conf:.2f}, Label = {label}")

            if found == 1:
                result.save(filename = os.path.join(r'C:\Users\Carlos Magahis\thesis\camera folder',f"image{int(time.time())}.png" )  )
                #result.plot(labels=True)
                new_path = os.path.join(r'C:\Users\Carlos Magahis\thesis\camera folder',f"image{int(time.time())}.png" )
                cv2.imwrite(new_path, result)
                dispimg = cv2.imread(new_path) 
                dispimg = cv2.cvtColor(dispimg, cv2.COLOR_RGB2BGR)
                self.image = dispimg
            else:
                dispimg = cv2.imread(image_path) 
                dispimg = cv2.cvtColor(dispimg, cv2.COLOR_RGB2BGR)
                self.image = dispimg
        if self.image is not None:
            image_resized = cv2.resize(self.image, (self.image_width, self.image_height))
            self.image_tk = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(image_resized))
            self.image_frame.create_image(0, 0, image=self.image_tk, anchor=tk.NW)
        else:
            self.image_frame.delete("all") 
        self.display_prediction(label)
    def display_prediction(self, label):
        self.text_output.delete("1.0", tk.END)
        self.text_output.insert(tk.END, f"Prediction:{label}\n")
        
    def select(self):
    # Open file selector and get image path
        image_path = filedialog.askopenfilename(initialdir="/", title="Select Image", filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
        
        

        if image_path:
          dispimg = cv2.imread(image_path)
          dispimg = cv2.cvtColor(dispimg, cv2.COLOR_RGB2BGR) #convert lang nung captured image
          self.image = dispimg
          self.predict_image(image_path)
        
        
    def update(self):
          ret, frame = self.vid.read()
          if ret:
              self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)))
              self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
          self.window.after(10, self.update)

# Create a window and pass it to the ImageCaptureApp class

app = ImageCaptureApp(root, "Diseased and Healthy Dragonfruit stem detector YOLOv9")


# In[ ]:







image 1/1 C:\Users\Carlos Magahis\thesis\dataset\test\anthracnose\image1731705297.png: 384x640 1 diseased, 1678.4ms
Speed: 209.2ms preprocess, 1678.4ms inference, 304.0ms postprocess per image at shape (1, 3, 384, 640)
None
Detection 0: Confidence = 0.78, Label = diseased


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\conda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Carlos Magahis\AppData\Local\Temp\ipykernel_40872\3337884679.py", line 168, in select
    self.predict_image(image_path)
  File "C:\Users\Carlos Magahis\AppData\Local\Temp\ipykernel_40872\3337884679.py", line 140, in predict_image
    dispimg = cv2.imread(new_path)
             ^^^^^^^^^^^^^^^^^^^^
  File "D:\conda\Lib\site-packages\ultralytics\utils\patches.py", line 26, in imread
    return cv2.imdecode(np.fromfile(filename, np.uint8), flags)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Carlos Magahis\\thesis\\camera folder\\image1739870558.png'
